# Dealing with PSDs/ASDs in Gravitational Wave Detectors

This notebook is a tutorial on how to deal with various PSDs (Power Spectral Densities) or ASDs (Amplitude spectral Density) in the context of gravitational wave detectors. 

## Contents

1. Default PSDs/ASDs in `gwsnr`

2. PSDs with GPS time

3. PSDs from .txt files
   
3. PSDs from `bilby`
   
4. PSDs from `pycbc`

## 1. Default PSDs/ASDs in `gwsnr`

`gwsnr` by default uses the following PSDs, which are also the default PSDs in `bilby`. It considers O4, observing run 4, design sensitivities. 

* L1, Livingston detector: `aLIGO_O4_high_asd.txt`
  
* H1, Hanford detector: `aLIGO_O4_high_asd.txt`
  
* V1, Virgo detector: `AdV_asd.txt`

In [10]:
from gwsnr import GWSNR
import numpy as np

In [11]:
gwsnr = GWSNR(snr_type='inner_product')

psds not given. Choosing bilby's default psds

Chosen GWSNR initialization parameters:

npool:  4
snr type:  inner_product
waveform approximant:  IMRPhenomD
sampling frequency:  2048.0
minimum frequency (fmin):  20.0
mtot=mass1+mass2
min(mtot):  2.0
max(mtot) (with the given fmin=20.0): 184.98599853446768
detectors:  ['L1', 'H1', 'V1']
psds:  [PowerSpectralDensity(psd_file='None', asd_file='/Users/phurailatpamhemantakumar/anaconda3/envs/ler/lib/python3.10/site-packages/bilby/gw/detector/noise_curves/aLIGO_O4_high_asd.txt'), PowerSpectralDensity(psd_file='None', asd_file='/Users/phurailatpamhemantakumar/anaconda3/envs/ler/lib/python3.10/site-packages/bilby/gw/detector/noise_curves/aLIGO_O4_high_asd.txt'), PowerSpectralDensity(psd_file='None', asd_file='/Users/phurailatpamhemantakumar/anaconda3/envs/ler/lib/python3.10/site-packages/bilby/gw/detector/noise_curves/AdV_asd.txt')]


In [12]:
# signal-to-noise ratio for a binary black hole merger with masses 10, 10 Msun at a luminosity distance of 100 Mpc
for psd in gwsnr.psds_list:
    print(psd)

PowerSpectralDensity(psd_file='None', asd_file='/Users/phurailatpamhemantakumar/anaconda3/envs/ler/lib/python3.10/site-packages/bilby/gw/detector/noise_curves/aLIGO_O4_high_asd.txt')
PowerSpectralDensity(psd_file='None', asd_file='/Users/phurailatpamhemantakumar/anaconda3/envs/ler/lib/python3.10/site-packages/bilby/gw/detector/noise_curves/aLIGO_O4_high_asd.txt')
PowerSpectralDensity(psd_file='None', asd_file='/Users/phurailatpamhemantakumar/anaconda3/envs/ler/lib/python3.10/site-packages/bilby/gw/detector/noise_curves/AdV_asd.txt')


In [13]:
# signal-to-noise ratio for a binary black hole merger with masses 10, 10 Msun at a luminosity distance of 100 Mpc
gwsnr.snr(mass_1=np.array([10.0]), mass_2=np.array([10.0]), luminosity_distance=np.array([100.0]))

solving SNR with inner product


100%|█████████████████████████████████████████████████████████████████| 1/1 [00:04<00:00,  4.61s/it]


{'L1': array([126.70353165]),
 'H1': array([80.58750192]),
 'V1': array([37.52436425]),
 'optimal_snr_net': array([154.77793225])}

## 2. PSDs with GPS time

The PSDs in `gwsnr` can be accessed with GPS time. Internally `gwsnr` uses `gwpy`'s `TimeSeries.fetch_open_data` attribute to download the PSDs. Fetched PSDs are stored in the `psd_data` directory and will be loaded from there if the same PSD is requested again.

I will get L1, H1 and V1 PSDs at GPS time 1187008682.4, which is 200s before the event GW170817. 16x32=512s of data will be downloaded starting from GPS time 1187008682.4-256=1187008426.4 to 1187008682.4.

In [15]:
from gwsnr import GWSNR
import numpy as np

gwsnr = GWSNR(
	psds=dict(
		L1=1187008682.4, # If loaded before, it will be called from the saved data (.txt file)
		H1=1187008682.4,  # If loaded before, it will be called from the saved data (.txt file)
	),
	snr_type='inner_product',
)

Loading psd data for L1 detector from ./psd_data/L1_1187008683_psd.txt
Loading psd data for H1 detector from ./psd_data/H1_1187008683_psd.txt

Chosen GWSNR initialization parameters:

npool:  4
snr type:  inner_product
waveform approximant:  IMRPhenomD
sampling frequency:  2048.0
minimum frequency (fmin):  20.0
mtot=mass1+mass2
min(mtot):  2.0
max(mtot) (with the given fmin=20.0): 184.98599853446768
detectors:  ['L1', 'H1']
psds:  [PowerSpectralDensity(psd_file='./psd_data/L1_1187008683_psd.txt', asd_file='None'), PowerSpectralDensity(psd_file='./psd_data/H1_1187008683_psd.txt', asd_file='None')]


In [16]:
# Print the list of PSD objects
for psd in gwsnr.psds_list:
    print(psd)

PowerSpectralDensity(psd_file='./psd_data/L1_1187008683_psd.txt', asd_file='None')
PowerSpectralDensity(psd_file='./psd_data/H1_1187008683_psd.txt', asd_file='None')


In [17]:
# signal-to-noise ratio for a binary black hole merger with masses 10, 10 Msun at a luminosity distance of 100 Mpc
gwsnr.snr(mass_1=np.array([10.0]), mass_2=np.array([10.0]), luminosity_distance=np.array([100.0]))

solving SNR with inner product


100%|█████████████████████████████████████████████████████████████████| 1/1 [00:04<00:00,  4.61s/it]


{'L1': array([66.59435999]),
 'H1': array([25.15463239]),
 'optimal_snr_net': array([71.18682682])}

## 3. PSDs from .txt files

Let's use the PSDs saved in the `psd_data` directory in the previous section. This file has two columns: frequency and PSD.

**Note**: PSD filename should end with `psd.txt` and ASD filename should end with `asd.txt`.

In [18]:
from gwsnr import GWSNR
import numpy as np

gwsnr = GWSNR(
	psds=dict(
		L1='./psd_data/L1_1187008683_psd.txt',
		H1='./psd_data/H1_1187008683_psd.txt',
		V1=1187008682.4,  # you can mix and match different types of psds/asds.If loaded before, it will be called from the saved data (.txt file)
	),
	snr_type='inner_product',
)

Loading psd data for V1 detector from ./psd_data/V1_1187008683_psd.txt

Chosen GWSNR initialization parameters:

npool:  4
snr type:  inner_product
waveform approximant:  IMRPhenomD
sampling frequency:  2048.0
minimum frequency (fmin):  20.0
mtot=mass1+mass2
min(mtot):  2.0
max(mtot) (with the given fmin=20.0): 184.98599853446768
detectors:  ['L1', 'H1', 'V1']
psds:  [PowerSpectralDensity(psd_file='./psd_data/L1_1187008683_psd.txt', asd_file='None'), PowerSpectralDensity(psd_file='./psd_data/H1_1187008683_psd.txt', asd_file='None'), PowerSpectralDensity(psd_file='./psd_data/V1_1187008683_psd.txt', asd_file='None')]


In [19]:
# Print the list of PSD objects
for psd in gwsnr.psds_list:
    print(psd)

PowerSpectralDensity(psd_file='./psd_data/L1_1187008683_psd.txt', asd_file='None')
PowerSpectralDensity(psd_file='./psd_data/H1_1187008683_psd.txt', asd_file='None')
PowerSpectralDensity(psd_file='./psd_data/V1_1187008683_psd.txt', asd_file='None')


In [20]:
# signal-to-noise ratio for a binary black hole merger with masses 10, 10 Msun at a luminosity distance of 100 Mpc
gwsnr.snr(mass_1=np.array([10.0]), mass_2=np.array([10.0]), luminosity_distance=np.array([100.0]))

solving SNR with inner product


100%|█████████████████████████████████████████████████████████████████| 1/1 [00:04<00:00,  4.83s/it]


{'L1': array([66.59435999]),
 'H1': array([25.15463239]),
 'V1': array([7.04454351]),
 'optimal_snr_net': array([71.53453646])}

## 4. PSDs from `bilby`

User can also use the PSDs or ASDs from `bilby`. Check for available PSDs/ASDs in `bilby` [here](https://github.com/bilby-dev/bilby/tree/master/bilby/gw/detector/noise_curves). Use the filename wit the extension as shown in the example below.

In [21]:
from gwsnr import GWSNR
import numpy as np

gwsnr = GWSNR(
	psds=dict(
		L1='aLIGO_O4_high_asd.txt',  # asd from bilby
		H1='./psd_data/H1_1187008683_psd.txt',  # psd from txt file
		V1=1187008682.4,  # psd from the give GPS time. If loaded before, it will be called from the saved data (.txt file)
	),
	snr_type='inner_product',
)

Loading psd data for V1 detector from ./psd_data/V1_1187008683_psd.txt

Chosen GWSNR initialization parameters:

npool:  4
snr type:  inner_product
waveform approximant:  IMRPhenomD
sampling frequency:  2048.0
minimum frequency (fmin):  20.0
mtot=mass1+mass2
min(mtot):  2.0
max(mtot) (with the given fmin=20.0): 184.98599853446768
detectors:  ['L1', 'H1', 'V1']
psds:  [PowerSpectralDensity(psd_file='None', asd_file='/Users/phurailatpamhemantakumar/anaconda3/envs/ler/lib/python3.10/site-packages/bilby/gw/detector/noise_curves/aLIGO_O4_high_asd.txt'), PowerSpectralDensity(psd_file='./psd_data/H1_1187008683_psd.txt', asd_file='None'), PowerSpectralDensity(psd_file='./psd_data/V1_1187008683_psd.txt', asd_file='None')]


In [22]:
# Print the list of PSD objects
for psd in gwsnr.psds_list:
    print(psd)

PowerSpectralDensity(psd_file='None', asd_file='/Users/phurailatpamhemantakumar/anaconda3/envs/ler/lib/python3.10/site-packages/bilby/gw/detector/noise_curves/aLIGO_O4_high_asd.txt')
PowerSpectralDensity(psd_file='./psd_data/H1_1187008683_psd.txt', asd_file='None')
PowerSpectralDensity(psd_file='./psd_data/V1_1187008683_psd.txt', asd_file='None')


In [23]:
# signal-to-noise ratio for a binary black hole merger with masses 10, 10 Msun at a luminosity distance of 100 Mpc
gwsnr.snr(mass_1=np.array([10.0]), mass_2=np.array([10.0]), luminosity_distance=np.array([100.0]))

solving SNR with inner product


100%|█████████████████████████████████████████████████████████████████| 1/1 [00:05<00:00,  5.19s/it]


{'L1': array([126.70353165]),
 'H1': array([25.15463239]),
 'V1': array([7.04454351]),
 'optimal_snr_net': array([129.36833483])}

## 5. PSDs from `pycbc`

User can use the PSDs or ASDs from `pycbc` package.

Check for available PSDs/ASDs in `pycbc` as shown below.

```python
    import pycbc.psd
    pycbc.psd.get_lalsim_psd_list()
```

In [1]:
# load pycbc first to avoid lal related  and multiprocessing issues
import warnings
warnings.filterwarnings("ignore", "Wswiglal-redir-stdio")
import pycbc

PyCBC.libutils: pkg-config call failed, setting NO_PKGCONFIG=1


In [2]:
from gwsnr import GWSNR
import numpy as np

gwsnr = GWSNR(
	psds={
		'L1':'aLIGOaLIGODesignSensitivityT1800044',
		'H1':'aLIGOaLIGODesignSensitivityT1800044',
		'V1':'AdvVirgo'
		},
	snr_type='inner_product',
)

Trying to get the psd from pycbc:  aLIGOaLIGODesignSensitivityT1800044
Trying to get the psd from pycbc:  aLIGOaLIGODesignSensitivityT1800044
Trying to get the psd from pycbc:  AdvVirgo

Chosen GWSNR initialization parameters:

npool:  4
snr type:  inner_product
waveform approximant:  IMRPhenomD
sampling frequency:  2048.0
minimum frequency (fmin):  20.0
mtot=mass1+mass2
min(mtot):  2.0
max(mtot) (with the given fmin=20.0): 184.98599853446768
detectors:  ['L1', 'H1', 'V1']
psds:  [PowerSpectralDensity(frequency_array=[0.0000000e+00 6.2500000e-02 1.2500000e-01 ... 2.0478125e+03
 2.0478750e+03 2.0479375e+03], psd_array=[0.00000000e+00 0.00000000e+00 0.00000000e+00 ... 1.06169994e-46
 1.06176023e-46 0.00000000e+00], asd_array=[0.00000000e+00 0.00000000e+00 0.00000000e+00 ... 1.03038825e-23
 1.03041750e-23 0.00000000e+00]), PowerSpectralDensity(frequency_array=[0.0000000e+00 6.2500000e-02 1.2500000e-01 ... 2.0478125e+03
 2.0478750e+03 2.0479375e+03], psd_array=[0.00000000e+00 0.00000000e+0

In [3]:
# Print the list of PSD objects
for psd in gwsnr.psds_list:
    print(psd)

PowerSpectralDensity(frequency_array=[0.0000000e+00 6.2500000e-02 1.2500000e-01 ... 2.0478125e+03
 2.0478750e+03 2.0479375e+03], psd_array=[0.00000000e+00 0.00000000e+00 0.00000000e+00 ... 1.06169994e-46
 1.06176023e-46 0.00000000e+00], asd_array=[0.00000000e+00 0.00000000e+00 0.00000000e+00 ... 1.03038825e-23
 1.03041750e-23 0.00000000e+00])
PowerSpectralDensity(frequency_array=[0.0000000e+00 6.2500000e-02 1.2500000e-01 ... 2.0478125e+03
 2.0478750e+03 2.0479375e+03], psd_array=[0.00000000e+00 0.00000000e+00 0.00000000e+00 ... 1.06169994e-46
 1.06176023e-46 0.00000000e+00], asd_array=[0.00000000e+00 0.00000000e+00 0.00000000e+00 ... 1.03038825e-23
 1.03041750e-23 0.00000000e+00])
PowerSpectralDensity(frequency_array=[0.0000000e+00 6.2500000e-02 1.2500000e-01 ... 2.0478125e+03
 2.0478750e+03 2.0479375e+03], psd_array=[0.00000000e+00 0.00000000e+00 0.00000000e+00 ... 2.44678421e-45
 2.44695644e-45 0.00000000e+00], asd_array=[0.00000000e+00 0.00000000e+00 0.00000000e+00 ... 4.94649796e-2

In [4]:
# signal-to-noise ratio for a binary black hole merger with masses 10, 10 Msun at a luminosity distance of 100 Mpc
gwsnr.snr(mass_1=np.array([10.0]), mass_2=np.array([10.0]), luminosity_distance=np.array([100.0]))

solving SNR with inner product


100%|█████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  2.56it/s]


{'L1': array([118.87799633]),
 'H1': array([75.61021096]),
 'V1': array([39.78184802]),
 'optimal_snr_net': array([146.39493654])}